# YOLO v8 training with Optuna

### Optuna is an open source hyperparameter optimization framework to automate hyperparameter search

### Intall dependencies

In [1]:
import os
HOME = os.path.split(os.getcwd())[0]
print(HOME)

e:\Code\CowId


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
!pip install ultralytics

In [4]:
!pip install optuna

In [2]:
# Check YOLO instalation

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.68  Python-3.8.16 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
Setup complete  (20 CPUs, 31.8 GB RAM, 403.1/465.5 GB disk)


In [3]:
# This is available for systems with nvidia GPUs to enable training accelaration with graphics cards:
# CUDA package: https://developer.nvidia.com/cuda-downloads

import torch
torch.cuda.is_available()

True

### Training with YOLO + Optuna

In [1]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="n8NPtdPwubKWNo8s10Yj")
# project = rf.workspace("cowid").project("cowid")
# dataset = project.version(10).download("yolov8")

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached wget-3.2-py3-none-any.whl
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
     ---------------------------------------- 0.0/59.3 kB ? eta -:--:--
     ---------------------------------------- 59.3/59.3 kB 3.1 MB/s eta 0:00:00
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
     ---------------------------------------- 0.0/38.0 MB ? eta -:--:--
     - -------------------------------------- 1.3/38.0 MB 28.1 MB/s eta 0:00:02
     --- ------------------------------------ 3.7/38.0 MB 39.5 MB/s eta 0:00:01
    

Extracting Dataset Version Zip to CowID-10 in yolov8:: 100%|██████████| 21044/21044 [00:04<00:00, 5170.89it/s]


In [5]:
import optuna
from ultralytics import YOLO

def objective(trial):
    # Load a model
    model = YOLO(f'{HOME}/yolov8s.pt') 
    # Set parameter ranges
    param_grid = {"epochs": trial.suggest_int("epochs", 10, 50, step=10), 
                  'lr': trial.suggest_float("lr", 0.01, 0.04, step=0.005), 
                  'dropout': trial.suggest_float("dropout", 0.0, 0.20),
                  'batch': trial.suggest_int("batch", 10, 50, step=10)}
    # Train with parameter suggestions
    model.train(data=f'{HOME}/data/CowID-10/data.yaml', 
                          epochs=param_grid['epochs'], 
                          batch=param_grid['batch'],  
                          plots=True, 
                          dropout=param_grid['dropout'],
                          lr0=param_grid['lr'],
                          seed=42)
 
    # Get validation results
    results = model.val() 
    precision = results.results_dict['metrics/precision(B)']
    recall = results.results_dict['metrics/recall(B)']
    return precision, recall

# Select to maximize or minimize parameters
study = optuna.create_study(directions=['maximize', 'maximize'])
study.optimize(objective, n_trials=8)

[I 2023-08-22 22:43:43,484] A new study created in memory with name: no-name-dac62712-75a1-4b8d-8b96-ec88115b2be1
New https://pypi.org/project/ultralytics/8.0.160 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.68  Python-3.8.16 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
yolo\engine\trainer: task=detect, mode=train, model=e:\Code\CowId/yolov8s.pt, data=e:\Code\CowId/data/CowID-10/data.yaml, epochs=30, patience=50, batch=20, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=42, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.04692730923924329, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, 

### Analyze the results

In [6]:
for trial in study.best_trials:
    print(f"Trial number: {trial.number}")
    print(trial.params)
    print(f"Trial precision: {trial.values[0]}, Trial recall: {trial.values[1]}")

Trial number: 0
{'epochs': 30, 'lr': 0.02, 'dropout': 0.04692730923924329, 'batch': 20}
Trial precision: 0.9408070855432032, Trial recall: 0.894484412470024
Trial number: 3
{'epochs': 20, 'lr': 0.01, 'dropout': 0.12815594143082534, 'batch': 20}
Trial precision: 0.9165687767078458, Trial recall: 0.9220810774269742
Trial number: 5
{'epochs': 40, 'lr': 0.03, 'dropout': 0.03707626298115357, 'batch': 30}
Trial precision: 0.9315023749140371, Trial recall: 0.8992805755395683


In [7]:
for trial in study.trials:
    print(f"Trial number: {trial.number}")
    print(trial.params)
    print(f"Trial precision: {trial.values[0]}, Trial recall: {trial.values[1]}")

Trial number: 0
{'epochs': 30, 'lr': 0.02, 'dropout': 0.04692730923924329, 'batch': 20}
Trial precision: 0.9408070855432032, Trial recall: 0.894484412470024
Trial number: 1
{'epochs': 10, 'lr': 0.04, 'dropout': 0.03759163356823643, 'batch': 40}
Trial precision: 0.8608081507049556, Trial recall: 0.8896882494004796
Trial number: 2
{'epochs': 10, 'lr': 0.015, 'dropout': 0.020805185909659588, 'batch': 10}
Trial precision: 0.8837260334040221, Trial recall: 0.8930913006717267
Trial number: 3
{'epochs': 20, 'lr': 0.01, 'dropout': 0.12815594143082534, 'batch': 20}
Trial precision: 0.9165687767078458, Trial recall: 0.9220810774269742
Trial number: 4
{'epochs': 10, 'lr': 0.02, 'dropout': 0.16160098256987615, 'batch': 50}
Trial precision: 0.8891680950057222, Trial recall: 0.8800959232613909
Trial number: 5
{'epochs': 40, 'lr': 0.03, 'dropout': 0.03707626298115357, 'batch': 30}
Trial precision: 0.9315023749140371, Trial recall: 0.8992805755395683
Trial number: 6
{'epochs': 10, 'lr': 0.03, 'dropout

In [8]:
# Save the parameters
import pickle

# save the list to a file
with open('trials2.pickle', 'wb') as f:
    pickle.dump(study.best_trials, f)

In [9]:
# load the list from the file
with open('trials2.pickle', 'rb') as f:
    trials = pickle.load(f)

### Prediction

In [2]:
!pip install opencv-python

In [1]:
from ultralytics import YOLO
from PIL import Image
import os
import cv2
import os
import numpy as np


HOME = os.path.split(os.getcwd())[0]
print(HOME)

WARNING  Ultralytics settings reset to defaults. This is normal and may be due to a recent ultralytics package update, but may have overwritten previous settings. 
View and update settings with 'yolo settings' or at 'C:\Users\arihs\AppData\Roaming\Ultralytics\settings.yaml'


d:\Code\CowId


In [2]:
model = YOLO(f"{HOME}/data/yolo_models/train11/weights/best.pt")

img_dir = os.path.join(HOME, 'data', 'videos_frames_2')

output_dir = os.path.join(HOME, 'data', 'yolo_output')

def expand_rectangle_to_square(x1, y1, x2, y2):
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    size = max(width, height)
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    new_x1 = center_x - size / 2
    new_y1 = center_y - size / 2
    new_x2 = center_x + size / 2
    new_y2 = center_y + size / 2
    return new_x1, new_y1, new_x2, new_y2

def crop_to_square(image_path, x1, y1, x2, y2):
    image = Image.open(image_path)
    cropped_image = image.crop((x1, y1, x2, y2))
    cropped_image = cropped_image.resize((250, 250))
    return cropped_image

In [3]:
for filename in os.listdir(img_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if filename.endswith(".jpg"):
        img_path = os.path.join(img_dir, filename)
        res = model(img_path, conf=0.5)
        boxes = res[0].boxes.xyxy.tolist()
        cow_number = 0
        for inner_list in boxes:
            cow_number += 1
            x1, y1, x2, y2 = inner_list
            x1, y1, x2, y2 = expand_rectangle_to_square(x1, y1, x2, y2)
            cropped_image = crop_to_square(img_path, x1, y1, x2, y2)
            output_filename = f"{os.path.splitext(filename)[0]}-{cow_number}.jpg"
            output_path = os.path.join(HOME, output_dir, output_filename)
            cropped_image.save(output_path)


image 1/1 D:\Code\CowId\data\videos_frames_2\video_cc_4-00000000.jpg: 384x640 (no detections), 84.1ms
Speed: 5.0ms preprocess, 84.1ms inference, 15.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Code\CowId\data\videos_frames_2\video_cc_4-00000002.jpg: 384x640 (no detections), 9.5ms
Speed: 1.0ms preprocess, 9.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Code\CowId\data\videos_frames_2\video_cc_4-00000004.jpg: 384x640 (no detections), 9.5ms
Speed: 0.0ms preprocess, 9.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Code\CowId\data\videos_frames_2\video_cc_4-00000006.jpg: 384x640 (no detections), 6.0ms
Speed: 0.0ms preprocess, 6.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Code\CowId\data\videos_frames_2\video_cc_4-00000008.jpg: 384x640 (no detections), 4.0ms
Speed: 0.0ms preprocess, 4.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 